In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tqdm
import numpy as np

In [3]:
X, y = make_regression(
    n_samples=1000,
    n_features=50,
    n_informative=20,
    random_state=0,
)

In [4]:
import torch

In [5]:
X = torch.from_numpy(X)
y = torch.from_numpy(y) / 100

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [11]:
class SquareLoss:
    def loss(self, y, y_pred):
        return 0.5 * ((y - y_pred)**2)

    def gradient(self, y, y_pred):
        return -(y - y_pred)

In [12]:
class GradientBoostingRegressor:
    def __init__(self, num_feature, lr=0.1, n_estimators=10, 
                 min_samples_split=2, min_impurity_decrease=0, max_depth=np.inf):
        self.num_feature = num_feature
        self.lr = lr
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.max_depth = max_depth
        self.loss = SquareLoss()
        
        self.trees = []
        for _ in range(self.n_estimators):
            self.trees.append(
                DecisionTreeRegressor(
                    num_feature, min_samples_split, 
                    min_impurity_decrease, max_depth
                )
            )
    
    def fit(self, X, y):
        y_pred = torch.zeros_like(y)
        for i in tqdm.tqdm(range(self.n_estimators)):
            gradient = self.loss.gradient(y, y_pred)
            self.trees[i].fit(X, gradient)
            update = self.trees[i].predict(X)
            # Update y prediction
            y_pred -= self.lr * update
    
    
    def predict(self, X):
        y_pred = torch.zeros(len(X))
        for tree in self.trees:
            update = tree.predict(X)
            update = self.lr * update
            y_pred = y_pred - update
        
        return y_pred

In [ ]:
clf = GradientBoostingRegressor(num_feature=50, lr=0.1, n_estimators=20, max_depth=5)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
err = mean_squared_error(y_test, pred)